In [1]:
from routines import reset_all_trades, create_new_investor, create_new_stock, print_all
from utils import get_net_worth, get_balance
from constants import name_id, id_name
import pandas as pd
from bank import buy_stock, sell_stock, pay_dividends
from routines import print_all
from utils import add_current_name_col
pd.set_option('display.width', 1800)


#### typical game steps

In [2]:
from prestige_hype import compute_prestige_and_hype


reset_all_trades()

create_new_investor('besta', 1000)
create_new_investor('carbone', 1000)
create_new_investor('mion1', 1000)

df = compute_prestige_and_hype()

for i,x in enumerate(df.index):
    pp,p,h = df.loc[x,:]
    create_new_stock(x, pp/1000, h, p, 1000, 0)




# pay_dividends('besta')
# pay_dividends('mion1')
# pay_dividends('carbone')

besta has just bought 3 share(s) of ThePooN for $510.24 !
carbone has just bought 1 share(s) of Zeaqk for $164.25 !
mion1 has just bought 2 share(s) of le ThibZ for $145.54 !


In [26]:
buy_stock('besta', 'ThePooN', 1)
# buy_stock('carbone', 'Zeaqk', 1)
# buy_stock('mion1', 'le ThibZ', 2)

besta does not have enough cash ($147.2) to perform this transaction ($171.79).


In [59]:
def tax_from_datetime(d):
    if d>30:
        return 0.2
    if d<30:
        return 0.1

In [74]:
def compute_tax_applied(trade_hist, quantity_to_sell):
    assert quantity_to_sell>0

    # 1-create stack where 1 layer = 1 quantity of shares owned and an associated datetime
    stack = []  # will contain only positive values
    for qty,tme in trade_hist:  # chronological order
        if qty>0:
            # add layer to stack
            stack.append([qty,tme])
        else:
            # "scrape off" layers from top to bottom
            qty_left = abs(qty)
            while qty_left > 0:
                top_layer_qty = stack[-1][0]
                if qty_left > top_layer_qty:  # remove this layer and decrease qty
                    qty_left -= top_layer_qty
                    del stack[-1]
                    
                else:  #modify value of top_layer
                    new_qty = top_layer_qty - qty_left
                    stack[-1][0] = new_qty
                    break   # break because we are done scraping
    
    # 2-Use stack to break down the quantity into smaller parts, with an associated datetime
    left_to_sell = quantity_to_sell
    parts=[]
    for qty,tme in reversed(stack):
        if qty<left_to_sell:
            left_to_sell -= qty
            parts.append([qty,tme])
        else:
            parts.append([left_to_sell,tme])
            break
    assert sum([x[0] for x in parts]) == quantity_to_sell
    
    # 4-calculate final tax%
    combination = [[x/abs(quantity_to_sell),tax_from_datetime(y)] for x,y in parts]
    
    res = sum([x*y for x,y in combination])  # linear combination
    return res


trade_hist = [[200,27],[-5,30],[102,45],[1,48],[-4,57],[-80,58],[3,59]]  #[qty, time]
quantity_to_sell = 40

compute_tax_applied(trade_hist, quantity_to_sell)

[[200, 27]]
[[195, 27]]
[[195, 27], [102, 45]]
[[195, 27], [102, 45], [1, 48]]
[[195, 27], [99, 45]]
[[195, 27], [19, 45]]
[[195, 27], [19, 45], [3, 59]]
final stack=[[195, 27], [19, 45], [3, 59]]
[[3, 59], [19, 45], [18, 27]]
[[0.075, 0.2], [0.475, 0.2], [0.45, 0.1]]


0.155

In [3]:
print_all()



STOCKS
             current_name  raw_skill  trendiness  prestige  total_shares  sold_shares  value_intrinsic   value  dividend_yield
name                                                                                                                          
8443945           FlasTEH    16.0317    1.751517  2.306913        1000.0          0.0            64.78   64.78            1.20
7657831   MBAPPEEEEEEEEEE    15.7357    1.838317  2.649792        1000.0          0.0            76.65   76.65            1.49
4141918           Thundur    15.5208    1.794019  2.752288        1000.0          0.0            76.64   76.64            1.56
14733798            Zeaqk    15.2386    5.223530  2.063442        1000.0          1.0           164.25  164.58            1.70
18323396    golem de caca    14.8554    2.061679  1.895878        1000.0          0.0            58.07   58.07            0.93
11948617           Saymel    14.7516    1.852964  1.962422        1000.0          0.0            53.64

In [23]:
from datetime import datetime
import time


history = pd.read_csv("transactions_history.csv", index_col='transaction_id')
history = history.astype({"stock_id": int})
history['datetime'] = pd.to_datetime(history['datetime'])
(datetime.now() - history.loc[0,'datetime']).seconds

45

In [5]:
# from routines import update_player_data_raw
# update_player_data_raw()

In [6]:
# print(add_current_name_col(df_raw))

In [7]:
# print_df = normalized_df.sort_values(by="computed_hype", ascending=False)

# print(print_df[['current_name','is_active','rank_current_to_mean','rank_current_to_worst','rank_current_to_highest_ever','is_silenced','scores_recent_count','computed_hype']])
# print(list(zip(list(print_df['current_name']),list(np.round(print_df['computed_hype'],2)))))

In [8]:
# from osuapi import api

# u = api.user(11843859)